# Spyralbound Architecture Design

## Overview
Spyralbound is a notebook backend library that uses Stryng for collaborative text editing while adding notebook-specific features like:
- **Entry Ordering**: Lexicographic keys for efficient cell reordering
- **Entry Types**: Code, markdown, data, output cells with type safety
- **Sandboxing**: Isolated execution environments for code cells
- **Document Structure**: Hierarchical organization and cross-references

## Design Philosophy
1. **Separation of Concerns**: Stryng handles text collaboration, Spyralbound handles notebook orchestration
2. **Performance First**: Efficient key management and minimal overhead
3. **Extensibility**: Plugin architecture for different cell types and execution engines
4. **Type Safety**: Full TypeScript support with proper interfaces

## 1. Import Required Libraries
Setting up dependencies for Spyralbound implementation

In [ ]:
// Core Stryng imports for collaborative text editing
import { Stryng, ChangeEvent, PersistenceChangeEvent } from 'stryng';
import * as Y from 'yjs';

// Utility imports for key generation and validation
import { nanoid } from 'nanoid';
import { z } from 'zod';

// Type definitions for Spyralbound
export interface SpyralboundConfig {
  docId: string;
  provider?: 'websocket' | 'supabase';
  persistence?: boolean;
  keyBase?: string; // Base for lexicographic keys (default: 'a')
  keyIncrement?: number; // Key generation increment (default: 65536)
}

export type EntryType = 
  | 'code'      // Executable code cells
  | 'markdown'  // Rich text documentation
  | 'data'      // Data visualization/tables
  | 'output'    // Code execution results
  | 'divider'   // Section separators
  | 'raw';      // Raw text/HTML

export type ExecutionState = 
  | 'idle'      // Not executed
  | 'running'   // Currently executing
  | 'completed' // Executed successfully
  | 'error';    // Execution failed

## 2. Define Entry Types and Constants
Core constants and validation schemas for notebook entries

In [ ]:
// Entry type constants and metadata
export const ENTRY_TYPES = {
  CODE: 'code',
  MARKDOWN: 'markdown', 
  DATA: 'data',
  OUTPUT: 'output',
  DIVIDER: 'divider',
  RAW: 'raw'
} as const;

// Execution environment constants
export const EXECUTION_ENGINES = {
  PYTHON: 'python',
  JAVASCRIPT: 'javascript',
  TYPESCRIPT: 'typescript',
  SQL: 'sql',
  SHELL: 'shell'
} as const;

// Validation schemas using Zod
export const EntrySchema = z.object({
  id: z.string(),
  type: z.enum(['code', 'markdown', 'data', 'output', 'divider', 'raw']),
  key: z.string(), // Lexicographic ordering key
  content: z.string(),
  metadata: z.record(z.any()).optional(),
  executionState: z.enum(['idle', 'running', 'completed', 'error']).optional(),
  executionEngine: z.string().optional(),
  createdAt: z.date(),
  modifiedAt: z.date()
});

export type Entry = z.infer<typeof EntrySchema>;

// Key generation constants for lexicographic ordering
export const KEY_CONFIG = {
  BASE: 'a',           // Starting character for keys
  INCREMENT: 65536,    // Large increment for efficient insertions
  MAX_DEPTH: 10,       // Maximum key nesting depth
  SEPARATOR: '.'       // Key component separator
} as const;

## 3. Implement Lexicographic Key Management
Efficient key generation for maintaining entry order without full document rewrite

In [ ]:
/**
 * Lexicographic Key Manager
 * Generates string keys that maintain sort order for efficient entry reordering
 */
export class LexKey {
  private static charCodeOffset = 97; // 'a' character code
  
  /**
   * Generate initial key for first entry
   */
  static initial(): string {
    return KEY_CONFIG.BASE;
  }
  
  /**
   * Generate key for inserting at the end
   */
  static append(lastKey: string): string {
    return lastKey + KEY_CONFIG.BASE;
  }
  
  /**
   * Generate key for inserting between two existing keys
   * Uses midpoint calculation to maintain lexicographic order
   */
  static between(keyA: string, keyB: string): string {
    if (keyA >= keyB) {
      throw new Error('keyA must be less than keyB');
    }
    
    // Find the first differing position
    let i = 0;
    while (i < Math.min(keyA.length, keyB.length) && keyA[i] === keyB[i]) {
      i++;
    }
    
    // If one key is a prefix of another
    if (i === keyA.length) {
      return keyA + KEY_CONFIG.BASE;
    }
    if (i === keyB.length) {
      return keyA.substring(0, i) + 
             String.fromCharCode((keyA.charCodeAt(i) + 122) / 2); // midpoint to 'z'
    }
    
    // Calculate midpoint character
    const charA = keyA.charCodeAt(i);
    const charB = keyB.charCodeAt(i);
    const midChar = Math.floor((charA + charB) / 2);
    
    return keyA.substring(0, i) + String.fromCharCode(midChar) + KEY_CONFIG.BASE;
  }
  
  /**
   * Generate key for inserting before existing key
   */
  static before(key: string): string {
    if (key === KEY_CONFIG.BASE) {
      return String.fromCharCode(KEY_CONFIG.BASE.charCodeAt(0) - 1);
    }
    return LexKey.between('', key);
  }
  
  /**
   * Validate key format and ordering
   */
  static validate(key: string): boolean {
    return /^[a-z]+$/.test(key) && key.length <= KEY_CONFIG.MAX_DEPTH;
  }
}

## 4. Create Base Entry Class
Foundation class for all notebook entries with Stryng integration

In [ ]:
/**
 * Base Entry Class
 * Represents a single notebook entry with Stryng text collaboration
 */
export class BaseEntry {
  public readonly id: string;
  public readonly type: EntryType;
  public key: string;
  public metadata: Record<string, any>;
  public executionState: ExecutionState;
  public executionEngine?: string;
  public readonly createdAt: Date;
  public modifiedAt: Date;
  
  // Stryng instance for collaborative text editing
  private stryng: Stryng;
  private changeCallbacks = new Set<(entry: BaseEntry) => void>();
  
  constructor(
    type: EntryType,
    key: string,
    initialContent = '',
    metadata: Record<string, any> = {}
  ) {
    this.id = nanoid();
    this.type = type;
    this.key = key;
    this.metadata = metadata;
    this.executionState = 'idle';
    this.createdAt = new Date();
    this.modifiedAt = new Date();
    
    // Create dedicated Stryng instance for this entry
    // Using entry ID as unique document identifier
    this.stryng = Stryng.create(initialContent, (change) => {
      this.modifiedAt = new Date();
      this.notifyChange();
    });
  }
  
  /**
   * Get current text content
   */
  getContent(): string {
    return this.stryng.get();
  }
  
  /**
   * Update text content
   */
  async setContent(content: string): Promise<void> {
    await this.stryng.update(content);
    this.modifiedAt = new Date();
  }
  
  /**
   * Subscribe to content changes
   */
  onChange(callback: (entry: BaseEntry) => void): () => void {
    this.changeCallbacks.add(callback);
    return () => this.changeCallbacks.delete(callback);
  }
  
  /**
   * Update entry metadata
   */
  updateMetadata(updates: Record<string, any>): void {
    this.metadata = { ...this.metadata, ...updates };
    this.modifiedAt = new Date();
    this.notifyChange();
  }
  
  /**
   * Set execution state
   */
  setExecutionState(state: ExecutionState, engine?: string): void {
    this.executionState = state;
    if (engine) this.executionEngine = engine;
    this.modifiedAt = new Date();
    this.notifyChange();
  }
  
  /**
   * Serialize entry to JSON
   */
  toJSON(): Entry {
    return {
      id: this.id,
      type: this.type,
      key: this.key,
      content: this.getContent(),
      metadata: this.metadata,
      executionState: this.executionState,
      executionEngine: this.executionEngine,
      createdAt: this.createdAt,
      modifiedAt: this.modifiedAt
    };
  }
  
  private notifyChange(): void {
    for (const callback of this.changeCallbacks) {
      callback(this);
    }
  }
  
  /**
   * Clean up resources
   */
  destroy(): void {
    this.stryng.destroy();
    this.changeCallbacks.clear();
  }
}

## 5. Implement Spyralbound Core Class
Main orchestrator class managing the entire notebook document structure

In [ ]:
/**
 * Spyralbound - Collaborative Notebook Backend
 * 
 * Core class that orchestrates notebook functionality while leveraging 
 * Stryng for collaborative text editing at the entry level.
 */
export class Spyralbound {
  private entries = new Map<string, BaseEntry>();
  private orderedKeys: string[] = [];
  private config: SpyralboundConfig;
  private changeCallbacks = new Set<(notebook: Spyralbound) => void>();
  
  // Document-level Stryng for metadata and structure
  private documentStryng?: Stryng;
  
  constructor(config: SpyralboundConfig) {
    this.config = { 
      keyBase: KEY_CONFIG.BASE,
      keyIncrement: KEY_CONFIG.INCREMENT,
      ...config 
    };
    
    // Optional: Use Stryng for document-level metadata synchronization
    if (config.persistence) {
      this.documentStryng = Stryng.create('{}', (change) => {
        // Handle document metadata changes
        this.notifyChange();
      });
    }
  }
  
  /**
   * Initialize with existing entries or create first entry
   */
  async initialize(initialEntries: Entry[] = []): Promise<void> {
    if (initialEntries.length === 0) {
      // Create initial markdown cell
      this.createEntry('markdown', 'Welcome to your notebook!');
    } else {
      // Load existing entries
      for (const entryData of initialEntries) {
        const entry = new BaseEntry(
          entryData.type,
          entryData.key,
          entryData.content,
          entryData.metadata
        );
        entry.setExecutionState(
          entryData.executionState || 'idle',
          entryData.executionEngine
        );
        
        this.entries.set(entry.id, entry);
        this.orderedKeys.push(entry.key);
        
        // Subscribe to entry changes
        entry.onChange(() => this.notifyChange());
      }
      
      // Sort keys to ensure proper ordering
      this.orderedKeys.sort();
    }
  }
  
  /**
   * Get all entries in order
   */
  getEntries(): BaseEntry[] {
    return this.orderedKeys
      .map(key => this.getEntryByKey(key))
      .filter((entry): entry is BaseEntry => entry !== undefined);
  }
  
  /**
   * Get entry by ID
   */
  getEntry(id: string): BaseEntry | undefined {
    return this.entries.get(id);
  }
  
  /**
   * Get entry by key (for ordering)
   */
  private getEntryByKey(key: string): BaseEntry | undefined {
    for (const entry of this.entries.values()) {
      if (entry.key === key) return entry;
    }
    return undefined;
  }
  
  /**
   * Create new entry at the end
   */
  createEntry(type: EntryType, content = '', metadata = {}): BaseEntry {
    const key = this.orderedKeys.length === 0 
      ? LexKey.initial()
      : LexKey.append(this.orderedKeys[this.orderedKeys.length - 1]);
      
    return this.createEntryWithKey(type, key, content, metadata);
  }
  
  /**
   * Insert entry at specific position
   */
  insertEntry(
    position: number, 
    type: EntryType, 
    content = '', 
    metadata = {}
  ): BaseEntry {
    let key: string;
    
    if (position === 0) {
      // Insert at beginning
      key = this.orderedKeys.length === 0 
        ? LexKey.initial()
        : LexKey.before(this.orderedKeys[0]);
    } else if (position >= this.orderedKeys.length) {
      // Insert at end
      key = LexKey.append(this.orderedKeys[this.orderedKeys.length - 1]);
    } else {
      // Insert between existing entries
      key = LexKey.between(
        this.orderedKeys[position - 1],
        this.orderedKeys[position]
      );
    }
    
    return this.createEntryWithKey(type, key, content, metadata);
  }
  
  private createEntryWithKey(
    type: EntryType, 
    key: string, 
    content: string, 
    metadata: Record<string, any>
  ): BaseEntry {
    const entry = new BaseEntry(type, key, content, metadata);
    
    this.entries.set(entry.id, entry);
    this.orderedKeys.push(key);
    this.orderedKeys.sort();
    
    // Subscribe to entry changes
    entry.onChange(() => this.notifyChange());
    
    this.notifyChange();
    return entry;
  }
  
  /**
   * Subscribe to notebook changes
   */
  onChange(callback: (notebook: Spyralbound) => void): () => void {
    this.changeCallbacks.add(callback);
    return () => this.changeCallbacks.delete(callback);
  }
  
  private notifyChange(): void {
    for (const callback of this.changeCallbacks) {
      callback(this);
    }
  }
  
  /**
   * Serialize entire notebook
   */
  toJSON() {
    return {
      config: this.config,
      entries: this.getEntries().map(entry => entry.toJSON()),
      createdAt: new Date(),
      version: '1.0.0'
    };
  }
  
  /**
   * Clean up all resources
   */
  destroy(): void {
    for (const entry of this.entries.values()) {
      entry.destroy();
    }
    this.entries.clear();
    this.orderedKeys = [];
    this.documentStryng?.destroy();
    this.changeCallbacks.clear();
  }
}

## Summary: Stryng Missing Features for Notebooks

Based on this Spyralbound design analysis, here are the key features Stryng needs for notebook support:

### **Critical Missing Features:**

1. **🎯 Multi-Text Document Architecture**
   - Current: One Y.Doc → One Y.Text → One Stryng instance
   - Needed: One Y.Doc → Multiple Y.Text instances → Shared sync/persistence

2. **🔍 Granular Change Events** 
   - Current: `{ source, value, delta }` (no text identification)
   - Needed: `{ source, textId, value, delta }` (which text changed)

3. **⚡ Selective Sync Policies**
   - Current: All changes sync to all users
   - Needed: Per-text sync configuration (local vs shared)

### **Recommended Stryng Enhancements:**

```typescript
// New multi-text API for notebooks
export class StryngDocument {
  private doc: Y.Doc;
  private texts = new Map<string, Y.Text>();
  
  createText(id: string, initialContent = ''): StryngText {
    const yText = this.doc.getText(id);
    return new StryngText(yText, id, this.onTextChange.bind(this));
  }
  
  private onTextChange(textId: string, change: ChangeEvent) {
    // Emit change with text identification
    this.emit('change', { ...change, textId });
  }
}

// Enhanced change events
type NotebookChangeEvent = ChangeEvent & {
  textId: string; // Which text instance changed
  syncPolicy?: 'local' | 'shared' | 'readonly';
};
```

### **Architecture Benefits:**
- ✅ **Atomic Sync**: All notebook cells sync together efficiently
- ✅ **Performance**: Single WebSocket/Supabase connection for entire notebook  
- ✅ **Conflict Resolution**: Y.js CRDTs handle concurrent edits across all cells
- ✅ **Selective Sharing**: Code cells sync, outputs stay local